In [20]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import subprocess
import cv2
%matplotlib inline

In [21]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.util import makeMovie

# スライド用画像作成

## 旋回中心と温度分布の混合

In [22]:
directories = ['20171108-37', '20171108-40', '20171108-43', '20171108-46']

In [43]:
for directory in directories:
    print(directory)
    out_dir = '/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171116/pvmap_temp/%s'%directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    vmem = RawCam('/mnt/recordings/SimulationResults/2D/%s/'%directory, cam_type='numpy',
           image_width=200, image_height=200, frame_start=0, frame_end=-1)
    vmem_temp = VmemMap(vmem)
    vmem.cmap = vmem_temp.cmap
    
    pmap = PhaseMap(vmem, shrink=2)
    pvmap = PhaseVarianceMap(pmap, size=3)
    
    for i in range(1000):
        num = "{0:04d}".format(i)
        temp = np.load('/mnt/recordings/SimulationResults/2D/%s/cell_%s/temp.npy' % (directory, num)).reshape((200,200))
        plt.imshow(temp, cmap='jet', alpha=0.5, vmin=302, vmax=310)
        #plt.imshow(cv2.resize((pvmap.data[i, :, :]>0.4).astype(np.uint8), (200,200)), cmap=pvmap.cmap, alpha=0.5, vmin=0, vmax=1)
        plt.imshow(cv2.resize(pvmap.data[i, :, :], (200,200)), cmap=pvmap.cmap, alpha=0.5, vmin=0, vmax=1)
        plt.xticks([])
        plt.yticks([])
        plt.text(6,16,"%s (ms)"% num,fontsize=16, color='white')

        plt.savefig('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171116/pvmap_temp/%s/%s.png' % (directory, num), dpi=300)
        plt.close()
    
    subprocess.call('ffmpeg -r 50 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171116/pvmap_temp/{0}/%04d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171116/pvmap_temp/{0}.mp4"'.format(directory), shell=True)

20171108-37
20171108-40
20171108-43
20171108-46
